In [50]:
!pip3 install langchain
!pip3 install OpenAI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
import os
import numpy as np
import random
import pandas as pd


from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = ""

In [52]:
INSTANCE_TYPE_NAME = "voter profiles"
N_GENERATED_EXAMPLES = 1

In [53]:
llm = ChatOpenAI(temperature=0.6)
template = """Given the following examples of {instance_type_name}, generate {n_generated_examples}
additional example for {instance_type_name}:

Example 1: {example_1}
Example 2: {example_2}
Example 3: {example_3}
Example 4: {example_4}
Example 5: {example_5}
Example 6: {example_6}
Example 7: {example_7}
Example 8: {example_8}
Example 9: 
"""

prompt = PromptTemplate(
    input_variables=["instance_type_name", "n_generated_examples", 
                     "example_1", "example_2","example_3", "example_4",
                     "example_5", "example_6", "example_7", "example_8"],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)

In [54]:
boot_exps = []
seed_data = pd.read_csv("/content/sample_data/seed.csv", header=None)
seed_examples = seed_data.iloc[:, 0].tolist()
for i in range(10):
  if i < 3:
    exps = random.sample(seed_examples, 8)
  else:
    rnd_seed_exps = random.sample(seed_examples, 6)
    rnd_boot_exps = random.sample(boot_exps, 2)
    exps = random.sample(rnd_seed_exps + rnd_boot_exps, 8)

  input_list = [INSTANCE_TYPE_NAME, N_GENERATED_EXAMPLES] + exps
  result = chain.run(instance_type_name=input_list[0], 
                     n_generated_examples=input_list[1],
                     example_1=input_list[2],
                     example_2=input_list[3],
                     example_3=input_list[4],
                     example_4=input_list[5],
                     example_5=input_list[6],
                     example_6=input_list[7],
                     example_7=input_list[8],
                     example_8=input_list[9])

  print(result)
  boot_exps.append(result)



Name: Ahmed. A 50-year-old Muslim man in Dearborn, Michigan, who identifies as a Democrat. He's a small business owner and concerned about economic inequality and access to healthcare. Ahmed is a strong advocate for immigrant rights and is critical of the current administration's policies on immigration. However, he can sometimes be dismissive of conservative religious beliefs, which can come off as intolerant. He values community involvement and social justice.
Name: Ahmed. A 30-year-old Muslim man from Detroit, Michigan, who identifies as a progressive Democrat. He works as a social worker and is passionate about racial justice and immigrant rights. Ahmed is critical of the criminal justice system and advocates for police reform. He supports universal healthcare and affordable housing, but his views on Israel and Palestine can be controversial within his community.
Name: Maria. A 55-year-old Hispanic woman from Miami, Florida, who is a moderate Democrat. She's a small business owner 

In [55]:
## To-do: generate multiple examples in the same call
## To-do: use techniques from Self-Instruct to clear out duplicates and bad examples 

In [56]:
out_df = pd.DataFrame(boot_exps)

# Save DataFrame as CSV without header
file_path = "out.csv" 
out_df.to_csv(file_path, header=False, index=False)